In [1]:
import numpy as np
import emcee
from scipy.integrate import quad

# Define the cosmological model
def luminosity_distance(z, H0, Omega_M, omega):
    c = 299792.458  # Speed of light in km/s
    integral = quad(lambda z_prime: 1.0 / np.sqrt(Omega_M*(1+z_prime)**3 + (1 - Omega_M - omega)*(1+z_prime)**2 + omega), 0, z)[0]
    return (1 + z) * c  * integral

def luminosity_to_mag(d_L):
    return 5 * np.log10(d_L)

# Likelihood function
def log_likelihood(params, z, observed_mag, observed_mag_err):
    H0, Omega_M, omega = params
    model_mag = luminosity_to_mag(np.array([luminosity_distance(z_i, H0, Omega_M, omega) for z_i in z]))
    chi2 = np.sum(((observed_mag - model_mag) / observed_mag_err) ** 2)
    return -0.5 * chi2

# Flat prior function
def log_prior(params):
    H0, Omega_M, omega = params
    if 50 < H0 < 100 and 0 < Omega_M < 1 and -2 < omega < 0:
        return 0.0  # log(1) = 0 for a flat prior
    return -np.inf  # log(0) = -inf for values outside the allowed ranges

# Posterior function
def log_posterior(params, z, observed_mag, observed_mag_err):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, z, observed_mag, observed_mag_err)

# Observed data (example redshifts and magnitudes)
observed_z = np.array([0.1, 0.5, 1.0, 1.5])
observed_mag = luminosity_to_mag(np.array([luminosity_distance(z, 70, 0.3, -1) for z in observed_z]))
observed_mag_err = np.ones_like(observed_mag) * 0.1  # Dummy uncertainties

# Set up the MCMC sampler
nwalkers = 16
ndim = 3
initial_guess = [70, 0.3, -1]
pos = initial_guess + 1e-4 * np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(observed_z, observed_mag, observed_mag_err))
sampler.run_mcmc(pos, 2000)

# Extract the samples
samples = sampler.get_chain(discard=500, thin=10, flat=True)

# Calculate the credible intervals
cred_int = np.percentile(samples, [16, 50, 84], axis=0)

# Print the credible intervals
print(f"H0: {cred_int[1, 0]:.2f} +{cred_int[2, 0] - cred_int[1, 0]:.2f} -{cred_int[1, 0] - cred_int[0, 0]:.2f}")
print(f"Omega_M: {cred_int[1, 1]:.3f} +{cred_int[2, 1] - cred_int[1, 1]:.3f} -{cred_int[1, 1] - cred_int[0, 1]:.3f}")
print(f"omega: {cred_int[1, 2]:.3f} +{cred_int[2, 2] - cred_int[1, 2]:.3f} -{cred_int[1, 2] - cred_int[0, 2]:.3f}")
